# Implementasi hasil VSM dengan algoritma Logistic Regression



memperbaiki load tfidf_vectorizer dan lr_model pada testing

* Nama  : Aurellia Zhullvita Amanullah
* NIM   : 220411100200
* Kelas : PPW A

# Import Library

In [31]:
!pip install -q Sastrawi

In [32]:
# library awal untuk perhitungan dan pengolahan teks
import numpy as np
import re
import pandas as pd

# alat untuk crawling
from urllib.request import urlopen
from bs4 import BeautifulSoup

# monitoring
from tqdm import tqdm

# library untuk praproses teks
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# library untuk proses modeling
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# library untuk evaluasi model
from sklearn.metrics import classification_report, confusion_matrix

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# save model
import pickle


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Persiapan Data

Load Data Model

In [33]:
main_df = pd.read_csv('https://raw.githubusercontent.com/aurelliazhullvita/kumpulan_tugas_ppw/refs/heads/master/data_beritakompas.csv', delimiter=',')
main_df

,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,Simak Jadwal dan Lokasi SIM Keliling di Jakart...,"JAKARTA, KOMPAS.com - Surat Izin Mengemudi (S...",07/09/2024,OTOMOTIF
1,2,[POPULER OTOMOTIF] Diskon Motor Honda Septembe...,"JAKARTA, KOMPAS.com - Banyak pembaca yang ingi...",07/09/2024,OTOMOTIF
2,3,"Cek Saldo Minimal BRI, BNI, BCA, Mandiri, dan BSI","JAKARTA, KOMPAS.com - Penting bagi calon nasab...",06/09/2024,MONEY
3,4,"KAI Uji Coba Teknologi ""Face Recognition Board...",KOMPAS.com - PT Kereta Api Indonesia (KAI) Div...,06/09/2024,MONEY
4,5,OJK Blokir 10.890 Entitas Keuangan Ilegal Seja...,"JAKARTA, KOMPAS.com - Otoritas Jasa Keuangan (...",06/09/2024,MONEY
...,...,...,...,...,...
95,96,Waspada Masalah yang Timbul akibat Telat Ganti...,"JAKARTA, KOMPAS.com - Oli mesin pada mobil den...",06/09/2024,OTOMOTIF
96,97,"Sosok Faisal Basri di Mata Para Tokoh, Ekonom ...","JAKARTA, KOMPAS.com - Ekonom senior Faisal Bas...",06/09/2024,MONEY
97,98,"Pendaftaran CPNS Diperpanjang 4 Hari, Pelamar ...","JAKARTA, KOMPAS.com - Pemerintah telah memperp...",06/09/2024,MONEY
98,99,"Harga Emas Terbaru Pegadaian, Jumat 6 Septembe...","JAKARTA, KOMPAS.com - Pegadaian menyediakan be...",06/09/2024,MONEY


# Membuat Fungsi untuk Persiapan Crawling

In [34]:
# fungsi untuk mengambil link yang akan dilakukan crawling
def extract_urls(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    urls = soup.find_all("a", {"class": "paging__link"})
    urls = [url.get('href') for url in urls]

    return urls

# fungsi untuk mengambil isi dari berita
def get_content(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, 'html.parser')

    div = soup.find("div", {"class": "read__content"})
    paragraf = div.find_all("p")

    content = ''
    for p in paragraf:
        content += p.text

    return content


# fungsi utama crawling
def crawl(link = "https://indeks.kompas.com", max_money = 1, max_otomotif = 1, allow_category = ["OTOMOTIF", "MONEY"], is_train = True, title_old = []):
    # inisialisasi variabel penampung hasil berita
    news_data = []

    # inisialisasi persiapan untuk crawling berita
    last_url = extract_urls(link).pop()
    page = last_url.split('=').pop() # jumlah halaman secara otomatis
    # page = 1 # jumlah halaman secara manual

    # persiapan link yang akan dilakukan crawling
    urls = [link + '/?page=' + str(a) for a in range(1, int(page) + 1)]
    count_money = 0
    count_otomotif = 0

    # menelusuri semua link yang telah ditentukan
    for idx, url in enumerate(urls):
        if (len(news_data) == max_money + max_otomotif) :
          break

        html = urlopen(url).read()
        soup = BeautifulSoup(html, 'html.parser')

        # mengambil data yang diperlukan pada struktur html
        links       = soup.find_all("a", {"class": "article-link"})
        titles      = soup.find_all("h2", {"class": "articleTitle"})
        dates       = soup.find_all("div", {"class": "articlePost-date"})
        categories  = soup.find_all("div", {"class": "articlePost-subtitle"})

        news_per_page = len(links) # berita artikel yang ditampilkan

        # memasukkan data ke dalam list
        for elem in tqdm(range(news_per_page), desc=f"Crawling page {idx+1}"):
          news = {}
          category = categories[elem].text
          title = titles[elem].text

          if (category in allow_category):
            if (is_train):
              cond = (category == "MONEY" and count_money < max_money) or (category == "OTOMOTIF" and count_otomotif < max_otomotif)
            else:
              cond = (category == "MONEY" and count_money < max_money) or (category == "OTOMOTIF" and count_otomotif < max_otomotif) and title not in title_old


            if (cond):
              news['No'] = len(news_data) + 1
              news['Judul Berita']     = title
              news['Isi Berita']       = get_content(links[elem].get("href"))
              news['Tanggal Berita']   = dates[elem].text
              news['Kategori Berita']  = category
              news_data.append(news)

              if (category == "MONEY"):
                count_money += 1
              else:
                count_otomotif += 1

        print(f"=======> Money: {count_money} | Otomotif: {count_otomotif} | Total: {count_money + count_otomotif}")

    return news_data

Fungsi extract_urls digunakan untuk mengekstraksi link URL dengan pagination di halaman awal, sehingga menghasilkan beberapa URL yang dapat menuju ke halaman berikutnya atau sebelumnya.

Fungsi get_content digunakan untuk menghasilkan konten berita berdasarkan link yang dicari.

# Pengambilan Data Baru

In [35]:
title_old = main_df["Judul Berita"].tolist()

test_news = crawl(max_money=5, max_otomotif=5, is_train=False, title_old=title_old)

Crawling page 1: 100%|██████████| 15/15 [00:02<00:00,  5.07it/s]


=======> Money: 2 | Otomotif: 2 | Total: 4


Crawling page 2: 100%|██████████| 15/15 [00:00<00:00, 16.38it/s]


=======> Money: 4 | Otomotif: 4 | Total: 8


Crawling page 3: 100%|██████████| 15/15 [00:00<00:00, 83.88it/s]


=======> Money: 5 | Otomotif: 4 | Total: 9


Crawling page 4: 100%|██████████| 15/15 [00:00<00:00, 82.53it/s]

=======> Money: 5 | Otomotif: 5 | Total: 10


In [36]:
main_df = pd.DataFrame(test_news)
main_df

,No,Judul Berita,Isi Berita,Tanggal Berita,Kategori Berita
0,1,MS Glow Berencana Bikin Akademi Balap Motor,"JAKARTA, KOMPAS.com - MS Glow for Man berencan...",26/09/2024,OTOMOTIF
1,2,Sensasi Berkendara Singkat All New Triton di M...,"JAKARTA, KOMPAS.com - Mitsubishi All New Trito...",26/09/2024,OTOMOTIF
2,3,Bus Listrik Akan Beroperasi di Jabar dan Sumba...,"JAKARTA, KOMPAS.com - Kementerian Perhubungan ...",26/09/2024,MONEY
3,4,"IHSG Kembali Tersungkur di Awal Perdagangan, R...","JAKARTA, KOMPAS.com - Indeks Harga Saham Gabun...",26/09/2024,MONEY
4,5,Alasan Kia Menghentikan Penjualan Varian Sonet...,"BANDUNG, KOMPAS.com – PT Kreta Indo Artha (KIA...",26/09/2024,OTOMOTIF
5,6,"Harga Emas Terbaru di Pegadaian Hari Ini, Kami...","JAKARTA, KOMPAS.com - Harga emas batangan Anta...",26/09/2024,MONEY
6,7,"Turun Rp 2.000, Harga Emas Antam Hari Ini: Det...","JAKARTA, KOMPAS.com - Pada Kamis 26 September ...",26/09/2024,MONEY
7,8,BAIC Siap Rakit Lokal BJ-40 mulai Maret 2025,"JAKARTA, KOMPAS.com - BAIC resmi memasuki pasa...",26/09/2024,OTOMOTIF
8,9,"Tahun Depan, Penumpang KRL Bisa Cek Kepadatan ...","JAKARTA, KOMPAS.com - PT Kereta Commuter Indon...",26/09/2024,MONEY
9,10,"Bagnaia Punya 2 Moge Ducati di Rumah, Standar ...","JAKARTA, KOMPAS.com - Jelang MotoGP Indonesia ...",26/09/2024,OTOMOTIF


# Membuat Fungsi

In [37]:
# Case Folding
def clean_lower(lwr):
    lwr = lwr.lower() # lowercase text
    return lwr

# Menghapus tanda baca, angka, dan simbol
def clean_punct(text):
    clean_spcl = re.compile('[/(){}\|@,;_]')
    clean_symbol = re.compile('[^0-9a-z]')
    clean_number = re.compile('[0-9]')
    text = clean_spcl.sub('', text)
    text = clean_symbol.sub(' ', text)
    text = clean_number.sub('', text)
    return text

# Menghaps double atau lebih whitespace
def _normalize_whitespace(text):
    corrected = str(text)
    corrected = re.sub(r"//t",r"\t", corrected)
    corrected = re.sub(r"( )\1+",r"\1", corrected)
    corrected = re.sub(r"(\n)\1+",r"\1", corrected)
    corrected = re.sub(r"(\r)\1+",r"\1", corrected)
    corrected = re.sub(r"(\t)\1+",r"\1", corrected)
    return corrected.strip(" ")

# Menghapus stopwords
def clean_stopwords(text):
    stopword = set(stopwords.words('indonesian'))
    text = ' '.join(word for word in text.split() if word not in stopword) # hapus stopword dari kolom deskripsi
    return text

# Stemming with Sastrawi
def sastrawistemmer(text):
    factory = StemmerFactory()
    st = factory.create_stemmer()
    text = ' '.join(st.stem(word) for word in tqdm(text.split()) if word in text)
    return text

Fungsi clean_lower digunakan untuk mengonversi semua kata atau huruf menjadi huruf kecil.

Fungsi clean_punct berfungsi untuk menghapus karakter khusus, simbol, dan angka.

Fungsi _normalize_whitespace digunakan untuk menghilangkan spasi ganda atau lebih dari dua spasi.

Fungsi clean_stopwords digunakan untuk menghapus kata-kata yang tidak relevan (seperti kata sambung atau kata tambahan).

Fungsi sastrawistemmer digunakan dalam proses stemming untuk mengubah kata ke bentuk dasarnya.

# Clean Lower

In [38]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses case folding
main_df['lwr'] = main_df['Isi Berita'].apply(clean_lower)
casefolding=pd.DataFrame(main_df['lwr'])
casefolding

,lwr
0,"jakarta, kompas.com - ms glow for man berencan..."
1,"jakarta, kompas.com - mitsubishi all new trito..."
2,"jakarta, kompas.com - kementerian perhubungan ..."
3,"jakarta, kompas.com - indeks harga saham gabun..."
4,"bandung, kompas.com – pt kreta indo artha (kia..."
5,"jakarta, kompas.com - harga emas batangan anta..."
6,"jakarta, kompas.com - pada kamis 26 september ..."
7,"jakarta, kompas.com - baic resmi memasuki pasa..."
8,"jakarta, kompas.com - pt kereta commuter indon..."
9,"jakarta, kompas.com - jelang motogp indonesia ..."


# Clean Punct

In [39]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan tanda baca
main_df['clean_punct'] = main_df['lwr'].apply(clean_punct)
main_df['clean_punct']


,clean_punct
0,jakarta kompas com ms glow for man berencana...
1,jakarta kompas com mitsubishi all new triton...
2,jakarta kompas com kementerian perhubungan k...
3,jakarta kompas com indeks harga saham gabung...
4,bandung kompas com pt kreta indo artha kia b...
5,jakarta kompas com harga emas batangan antam...
6,jakarta kompas com pada kamis september ha...
7,jakarta kompas com baic resmi memasuki pasar...
8,jakarta kompas com pt kereta commuter indone...
9,jakarta kompas com jelang motogp indonesia y...


# Normalize Whitespace

In [40]:
main_df['clean_double_ws'] = main_df['clean_punct'].apply(_normalize_whitespace)
main_df['clean_double_ws']

,clean_double_ws
0,jakarta kompas com ms glow for man berencana m...
1,jakarta kompas com mitsubishi all new triton r...
2,jakarta kompas com kementerian perhubungan kem...
3,jakarta kompas com indeks harga saham gabungan...
4,bandung kompas com pt kreta indo artha kia bar...
5,jakarta kompas com harga emas batangan antam p...
6,jakarta kompas com pada kamis september harga ...
7,jakarta kompas com baic resmi memasuki pasar i...
8,jakarta kompas com pt kereta commuter indonesi...
9,jakarta kompas com jelang motogp indonesia yan...


# Clean Stopwords

In [41]:
# Buat kolom tambahan untuk data description yang telah dilakukan proses penghapusan stopwords
main_df['clean_sw'] = main_df['clean_double_ws'].apply(clean_stopwords)
main_df['clean_sw']

,clean_sw
0,jakarta kompas com ms glow for man berencana a...
1,jakarta kompas com mitsubishi all new triton r...
2,jakarta kompas com kementerian perhubungan kem...
3,jakarta kompas com indeks harga saham gabungan...
4,bandung kompas com pt kreta indo artha kia mer...
5,jakarta kompas com harga emas batangan antam t...
6,jakarta kompas com kamis september harga emas ...
7,jakarta kompas com baic resmi memasuki pasar i...
8,jakarta kompas com pt kereta commuter indonesi...
9,jakarta kompas com jelang motogp indonesia sir...


# Stemming dengan Sastrawi

In [42]:
# Buat kolom tambahan untuk data description yang telah dilemmatization
main_df['desc_clean_stem'] = main_df['clean_sw'].apply(sastrawistemmer)
main_df['desc_clean_stem']


100%|██████████| 126/126 [00:05<00:00, 23.80it/s]


,desc_clean_stem
0,jakarta kompas com ms glow for man rencana aka...
1,jakarta kompas com mitsubishi all new triton r...
2,jakarta kompas com menteri hubung kemenhub lay...
3,jakarta kompas com indeks harga saham gabung i...
4,bandung kompas com pt kreta indo artha kia ril...
5,jakarta kompas com harga emas batang antam cat...
6,jakarta kompas com kamis september harga emas ...
7,jakarta kompas com baic resmi pasuk pasar indo...
8,jakarta kompas com pt kereta commuter indonesi...
9,jakarta kompas com jelang motogp indonesia sir...


# VSM

In [43]:
# Load the saved model from file
filename = 'tfidf_vectorizer.sav'
tfidf_vectorizer = pickle.load(open(filename, 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [44]:
corpus = main_df['desc_clean_stem']
tfidf = tfidf_vectorizer.transform(corpus)

In [45]:
vocabulary = tfidf_vectorizer.get_feature_names_out().tolist()

tfidf_df = pd.DataFrame(tfidf.toarray(), columns=vocabulary)
tfidf_df.insert(0, 'Kategori Berita', main_df['Kategori Berita'])
tfidf_df

,Kategori Berita,aaion,aali,abadi,abai,abenkh,abnormal,absurd,ac,acapkali,...,za,zad,zag,zaman,zarco,zenix,zero,zig,zigzag,zona
0,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.152944
4,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,MONEY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,OTOMOTIF,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


# Preparing Data

# Encode Label

Encode Label adalah proses mengubah data kategorik (seperti teks atau label) menjadi bentuk numerik sehingga bisa digunakan dalam model machine learning. Ini bertujuan agar algoritma dapat memahami dan memproses data tersebut.

In [46]:
# menggunakan label_encoder untuk merubah kata menjadi angka
label_encoder = preprocessing.LabelEncoder()
tfidf_df['Kategori Berita'] = label_encoder.fit_transform(tfidf_df['Kategori Berita'])

tfidf_df

,Kategori Berita,aaion,aali,abadi,abai,abenkh,abnormal,absurd,ac,acapkali,...,za,zad,zag,zaman,zarco,zenix,zero,zig,zigzag,zona
0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.152944
4,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


# Testing Data

In [47]:
# Load the saved model from file
filename = 'lr_model.sav'
lr_model = pickle.load(open(filename, 'rb'))

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.5.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [48]:
y_test = tfidf_df['Kategori Berita']
x_test = tfidf_df.drop(['Kategori Berita'], axis=1)
y_pred = lr_model.predict(x_test)

print(y_pred)

[1 1 1 0 1 0 0 1 0 1]


In [49]:
# melihat nilai actual dan predicted
a = pd.DataFrame({'Actual value': y_test, 'Predicted value':y_pred})
a

,Actual value,Predicted value
0,1,1
1,1,1
2,0,1
3,0,0
4,1,1
5,0,0
6,0,0
7,1,1
8,0,0
9,1,1
